In [2]:
import pickle
import plotly.express as px
import numpy as np
import pandas as pd
import re

# **BIG 5를 측정하는 Glove 모델을 Import**



In [6]:
cEXT = pickle.load( open(  "./models/cEXT.p", "rb"), buffers=None)
cNEU = pickle.load( open(  "./models/cNEU.p", "rb"), buffers=None)
cAGR = pickle.load( open(  "./models/cAGR.p", "rb"), buffers=None)
cCON = pickle.load( open(  "./models/cCON.p", "rb"), buffers=None)
cOPN = pickle.load( open(  "./models/cOPN.p", "rb"), buffers=None)
vectorizer_31 = pickle.load( open(  "./models/vectorizer_31.p", "rb"), buffers=None)
vectorizer_30 = pickle.load( open(  "./models/vectorizer_30.p", "rb"), buffers=None)

**Big 5 지표를 기반으로 한 Glove 모델을 MBTI로 변환**

In [7]:
def predict_personality(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    text_vector_30 = vectorizer_30.transform(scentences)
    EXT = cEXT.predict(text_vector_31)
    # NEU = cNEU.predict(text_vector_30)
    AGR = cAGR.predict(text_vector_31)
    CON = cCON.predict(text_vector_31)
    OPN = cOPN.predict(text_vector_31)
    List = []
    if EXT[0] == 1:
      List.append('E')
    else:
      List.append('I')
    if OPN[0] == 1:
      List.append('N')
    else:
      List.append('S')
    if AGR[0] == 1:
      List.append('F')
    else:
      List.append('T')
    if CON[0] == 1:
      List.append('J')
    else:
      List.append('P')
    return List

# **게임 데이터 셋 Import**

# 1. 전처리

In [8]:
data=pd.read_csv('steam_games.csv',index_col='name')

**좋은 리뷰의 비율, 장르, 게임 설명을 제외한 Column 삭제**

In [9]:
data2=data.loc[:,['all_reviews','genre','game_description']]
data2=data2.dropna()

In [10]:
data2

,all_reviews,genre,game_description
name,,,
DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...",Action,"About This Game Developed by id software, the..."
PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...
BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...
DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...
EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game
...,...,...,...
A Room Beyond,"Positive,(11),- 90% of the 11 user reviews for...","Adventure,Indie","About This Game A ROOM BEYOND, is fantasy poi..."
Die In The Dark,"1 user reviews,- Need more user reviews to gen...","Adventure,Indie,Simulation",About This Game Die in the Dark is a first-pe...
Chasing the Stars,"Mostly Positive,(10),- 70% of the 10 user revi...",Indie,About This Game After building some satellite...


 **좋은 리뷰의 비율이 70% 이상인 것만 추출하기 위해, 
all reviews를 각각의 % 값으로 대체** 

In [11]:
new_num=[]
for num in data2['all_reviews']:
    i=0
    num1=str(num)
    num2=num1.split()
    for j in num2:
      if '%' in j:
        new_num.append(j)
        i=1
    if i==0:
      data2[data2['all_reviews']==num] = np.NAN

In [12]:
data2=data2.dropna()

In [13]:
new_num2=[]
for j in new_num:
    j=j.strip('%')
    j=int(j)
    new_num2.append(j)
# print(new_num2)

# 평균값 중간값 추출
print(np.mean(new_num2))

74.62578397212543


In [14]:
len(new_num2)

17220

# 2. 게임의 성격 추출

 **장르 열에 해당하는 내용을 '장르+게임 설명'의 내용으로 대체하여 new_gen2의 리스트에 넣고, Personality 추출:**

In [15]:
new_gen2=[]
for j in range(len(data2['genre'])):
    gen2=str(data2['genre'][j])+str(data2['game_description'][j])
    new_gen2.append(gen2)
print(new_gen2[1])

Action,Adventure,Massively Multiplayer About This Game  PLAYERUNKNOWN'S BATTLEGROUNDS is a battle royale shooter that pits 100 players against each other in a struggle for survival. Gather supplies and outwit your opponents to become the last person standing. PLAYERUNKNOWN , aka Brendan Greene, is a pioneer of the battle royale genre and the creator of the battle royale game modes in the ARMA series and H1Z1: King of the Kill. At PUBG Corp., Greene is working with a veteran team of developers to make PUBG into the world's premiere battle royale experience.


In [16]:
pre_list=[]
for text in new_gen2[0:]:
  predictions = predict_personality(text)
  pre_list.append(predictions)

**구한 게임의 Personality와, 좋은 리뷰의 비율을 데이터프레임에 반영**

In [17]:
for i in range(len(new_gen2)):
        data2['genre'][i]=pre_list[i]       

for j in range(len(new_num2)):
        data2['all_reviews'][j]=new_num2[j]

data2.head()

/var/folders/x4/z7w83zkj1q92z10zj7s__cxc0000gn/T/ipykernel_41049/2805588284.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['genre'][i]=pre_list[i]
/var/folders/x4/z7w83zkj1q92z10zj7s__cxc0000gn/T/ipykernel_41049/2805588284.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['all_reviews'][j]=new_num2[j]


,all_reviews,genre,game_description
name,,,
DOOM,92,"[E, N, F, J]","About This Game Developed by id software, the..."
PLAYERUNKNOWN'S BATTLEGROUNDS,49,"[E, N, F, P]",About This Game PLAYERUNKNOWN'S BATTLEGROUND...
BATTLETECH,71,"[E, N, T, J]",About This Game From original BATTLETECH/Mec...
DayZ,61,"[E, N, T, J]",About This Game The post-soviet country of Ch...
EVE Online,74,"[E, N, T, P]",About This Game


In [18]:
#data3 = (data2.recent_reviews >= int(70)) 
#data4 = data2.loc[data3, :]
#print(data4.head())

#data2['all_reviews']=data2['all_reviews'].astype(int)
data3=data2[data2['all_reviews']>70]

In [19]:
data3=data3.rename(columns={'genre':'MBTI','all_reviews':'All_good_reviews_ratio'})

In [20]:
data3

,All_good_reviews_ratio,MBTI,game_description
name,,,
DOOM,92,"[E, N, F, J]","About This Game Developed by id software, the..."
BATTLETECH,71,"[E, N, T, J]",About This Game From original BATTLETECH/Mec...
EVE Online,74,"[E, N, T, P]",About This Game
Devil May Cry 5,92,"[E, N, T, J]",About This Game The Devil you know returns in...
Human: Fall Flat,91,"[E, N, T, P]","About This Game ***NEW ""DARK"" LEVEL AVAILABL..."
...,...,...,...
雨上がりのハナビィ Ameagari no Hanaby,100,"[I, N, F, J]",About This Game Ameagari no Hanaby is a sing...
Lil Big Invasion,100,"[E, N, T, J]",About This Game Lil Big Invasion is an explor...
Grabity,100,"[E, N, F, P]",About This Game Welcome to Grabity's ballist...


**좋은 비율이 많은 순서로 추천을 진행하기 위해, 내림차순으로 정리함 + type을 string으로 변경**

In [21]:
data3=data3.sort_values(by='All_good_reviews_ratio', ascending=False)

In [22]:
#data4=data3['(EXT, NEU, AGR, CON, OPN)'].str.contains('[1, 1, 1, 1, 1]')
data3['MBTI']=data3['MBTI'].astype(str)
data3

,All_good_reviews_ratio,MBTI,game_description
name,,,
Nowhere Station,100,"['E', 'N', 'T', 'P']",About This Game As a Station flies into a mys...
Ship It,100,"['E', 'N', 'F', 'P']",About This Game Fast-paced block stacking VR ...
Lupinball,100,"['I', 'N', 'T', 'P']",About This Game Werewolves shooting fireball...
100% Orange Juice - Mixed Booster Pack,100,"['E', 'N', 'T', 'P']",About This Content Mixed Booster Pack include...
HUNTER BRICK BALL,100,"['E', 'N', 'F', 'P']",About This Game Perfect time killer! Hunter b...
...,...,...,...
Chess Ultra,71,"['I', 'N', 'T', 'P']",About This Game Introducing the most breathta...
Just Cause 2: Chevalier Classic,71,"['E', 'N', 'T', 'P']",About This Content No agent should be without...
ELEX,71,"['I', 'N', 'T', 'J']",About This Game ELEX is a handcrafted action ...


# 유저의 성격 추출

**추천된 영화와 책의 줄거리를 텍스트로 넣어, User 2542의 성격 추출**

In [28]:
predictions = predict_personality(" The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son. The Godfather 'Don' Vito Corleone is the head of the Corleone mafia family in New York. He is at the event of his daughter's wedding. Michael, Vito's youngest son and a decorated WW II Marine is also present at the wedding. Michael seems to be uninterested in being a part of the family business. Vito is a powerful man, and is kind to all those who give him respect but is ruthless against those who do not. But when a powerful and treacherous rival wants to sell drugs and needs the Don's influence for the same, Vito refuses to do it. What follows is a clash between Vito's fading old values and the new ways which may cause Michael to do the thing he was most reluctant in doing and wage a mob war against all the other mafia families which could tear the Corleone family apart. In late summer 1945, guests are gathered for the wedding reception of Don Vito Corleone's daughter Connie and Carlo Rizzi . Vito , the head of the Corleone Mafia family, is known to friends and associates as 'Godfather.' He and Tom Hagen, the Corleone family lawyer, are hearing requests for favors because, according to Italian tradition, 'no Sicilian can refuse a request on his daughter's wedding day.' One of the men who asks the Don for a favor is Amerigo Bonasera, a successful mortician and acquaintance of the Don, whose daughter was brutally beaten by two young men because she refused their advances; the men received minimal punishment from the presiding judge. The Don is disappointed in Bonasera, who'd avoided most contact with the Don due to Corleone's nefarious business dealings. The Don's wife is godmother to Bonasera's shamed daughter, a relationship the Don uses to extract new loyalty from the undertaker. The Don agrees to have his men punish the young men responsible in a non-lethal manner in return for future service if necessary. Meanwhile, the Don's youngest son Michael ,a decorated US Marine hero returning from World War II service, arrives at the wedding and tells his girlfriend Kay Adams anecdotes about his family, informing her about his father's criminal life; he reassures her that he is different from his family and doesn't plan to join them in their criminal dealings. The wedding scene serves as critical exposition for the remainder of the film, as Michael introduces the main characters to Kay. Fredo, Michael's next older brother, is a bit dim-witted and quite drunk by the time he finds Michael at the party. Santino, who is nicknamed Sonny , the Don's eldest child and next in line to become Don upon his father's retirement, is married but he is a hot-tempered philanderer who sneaks into a bedroom to have sex with one of Connie's bridesmaids, Lucy Mancini . Tom Hagen is not related to the family by blood but is considered one of the Don's sons because he was homeless when he befriended Sonny in the Little Italy neighborhood of Manhattan and the Don took him in and saw to Tom's upbringing and education. Now a talented attorney, Tom is being groomed for the important position of consigliere (counselor) to the Don, despite his non-Sicilian heritage. Also among the guests at the celebration is the famous singer Johnny Fontane ,Corleone's godson, who has come from Hollywood to petition Vito's help in landing a movie role that will revitalize his flagging career. Jack Woltz, the head of the studio, denies Fontane the part (a character much like Johnny himself), which will make him an even bigger star, but Don Corleone explains to Johnny: 'I'm gonna make him an offer he can't refuse.' The Don also receives congratulatory salutations from Luca Brasi, a terrifying enforcer in the criminal underworld, and fills a request from the baker, Nazorine, who made Connie's wedding cake who wishes for his nephew Enzo to become an American citizen. After the wedding, Hagen is dispatched to Los Angeles to meet with Woltz, but Woltz angrily tells him that he will never cast Fontane in the role. Woltz holds a grudge because Fontane seduced and 'ruined' a starlet who Woltz had been grooming for stardom and with whom he had a sexual relationship. Woltz is persuaded to give Johnny the role, however, when he wakes up early the next morning and feels something wet in his bed. He pulls back the sheets and finds himself in a pool of blood; he screams in horror when he discovers the severed head of his prized $600,000 stud horse, Khartoum, in the bed with him. (A deleted scene from the film implies that Luca Brasi , Vito's top 'button man' or hitman, is responsible.) Upon Hagen's return, the family meets with Virgil 'The Turk' Sollozzo , who is being backed by the rival Tattaglia family. He asks Don Corleone for financing as well as political and legal protection for importing and distributing heroin. Despite the huge profit to be made, Vito Corleone refuses, explaining that his political influence would be jeopardized by a move into the narcotics trade -- the judges and politicians he's allied himself with over the course of several decades would renounce their friendships with him if he were to enter the drug trade. The Don's eldest son, Sonny, who had earlier urged the family to enter the narcotics trade, breaks rank during the meeting and begins to question Sollozzo's assurances as to the Corleone Family's investment being guaranteed by the Tattaglia Family. His father, angry at Sonny's dissension in a non-family member's presence, silences Sonny with a single look and privately rebukes him later. Don Corleone then dispatches Luca Brasi to infiltrate Sollozzo's organization and report back with information. During the meeting, while Brasi is bent over to allow Bruno Tattaglia to light his cigarette, he is stabbed in the hand by Sollozzo, and is subsequently garroted by an assassin. soon after his meeting with Sollozzo, Don Corleone is gunned down in an assassination attempt just outside his office, and it is not immediately known whether he has survived. Fredo Corleone had been assigned driving and protection duty for his father when Paulie Gatto, the Don's usual bodyguard, had called in sick. Fredo proves to be ineffectual, fumbling with his gun and unable to shoot back. When Sonny hears about the Don being shot and Paulie's absence, he orders Clemenza , one of his father's two 'caporegimes,' to find Paulie and bring him to the Don's house. Sollozzo abducts Tom Hagen and holds him for several hours, persuading him to offer Sonny the deal previously offered to his father. When Tom is released, Sollozzo gets word that the Don has survived the attempt on his life. He angrily tells Tom to convince Sonny to accept his offer. Enraged, Sonny refuses to consider it and issues an ultimatum to the Tattaglias: turn over Sollozzo or face a lengthy, bloody and costly (for both sides) gang war. They refuse, and instead send Sonny 'a Sicilian message,' in the form of two fresh fish wrapped in Luca Brasi's bullet-proof vest, telling the Corleones that Luca Brasi 'sleeps with the fishes.' Clemenza later takes Paulie and one of the family's hitmen, Rocco Lampone, for a drive into Manhattan. Sonny wants to 'go to the mattresses' -- set up beds in apartments for Corleone button men to operate out of in the event that the crime war breaks out. On their way back from Manhattan, Clemenza has Paulie stop the car in a remote area so he can urinate. Rocco shoots Paulie dead; he and Clemenza leave Paulie and the car behind. Michael, whom the other Mafia families consider a 'civilian' and not involved in mob business, visits his father at a small private hospital after having dinner with Kay at her hotel. He is shocked to find that no one is guarding him -- a nurse tells him that the men were interfering with hospital policy and were told to leave by the police about 10 minutes before Mike's arrival. Realizing that his father is again being set up to be killed, he calls Sonny for help, moves his father to another room, and goes outside to watch the entrance. Michael enlists help from Enzo the baker , who has come to the hospital to pay his respects. Together, they bluff away Sollozzo's men as they drive by. Police cars soon appear bringing the corrupt Captain McCluskey , who viciously punches Michael in the cheek and breaks his jaw when Michael insinuates that Sollozzo paid McCluskey to set up his father. Just then, Hagen arrives with 'private detectives' licensed to carry guns to protect Don Corleone, and he takes the injured Michael home. Sonny responds by having Bruno Tattaglia , the eldest son and underboss of Don Phillip Tattaglia , killed (off-camera). Following the attempt on the Don's life at the hospital, Sollozzo requests a meeting with the Corleones, which Captain McCluskey will attend as Sollozzo's bodyguard. When Michael volunteers to kill both men during the meeting, Sonny and the other senior Family members are amused; however, Michael convinces them that he is serious and that killing Sollozzo and McCluskey is in the family's interest: 'It's not personal. It's strictly business.' Because Michael is considered a civilian, he won't be regarded as a suspicious ambassador for the Corleones. Although police officers are usually off limits for hits, Michael argues that since McCluskey is corrupt and has illegal dealings with Sollozzo, he is fair game. Michael also implies that newspaper reporters that the Corleones have on their payroll would delight in publishing stories about a corrupt police captain. Michael meets with Clemenza, who prepares a small pistol for him, covering the trigger and grip with tape to prevent any fingerprint evidence. He instructs Michael about the proper way to perform the assassination and tells him to leave the gun behind. He also tells Michael that the family were all very proud of Michael for becoming a war hero during his service in the Marines and that a war like the impending one that Sollozzo's and McClusky's killings will spark is necessary about every five to tens years to clean out the ambition and resentment that builds between the Five Families. Clemenza shows great confidence that Michael can perform the job and tells him it will all go smoothly. The plan is to have the Corleone's informers find out the location of the meeting and plant the revolver before Michael, Sollozzo and McCluskey arrive. Before he leaves for the meeting, Sonny tells Michael he'll get word to Kay about not saying goodbye. Before the meeting in a small Italian restaurant in the Bronx, McCluskey frisks Michael for weapons and finds him clean. After a few minutes where Michael and Sollozzo converse in Italian, Michael excuses himself to go to the bathroom, where he retrieves the planted revolver. Returning to the table, he fatally shoots Sollozzo, then McCluskey. Michael is sent to hide in Sicily while the Corleone family prepares for all-out warfare with the Five Families (who are united against the Corleones) as well as a general clampdown on the mob by the police and government authorities. Three months later, when the don returns home from the hospital, he is distraught to learn that it was Michael who killed Sollozzo and McCluskey. Meanwhile, Connie and Carlo's marriage is disintegrating. They argue frequently over Carlo's suspected infidelity and his possessive behavior toward Connie. By Italian tradition, nobody, not even a high-ranking Mafia don, can intervene in a married couple's personal disputes, even if they involve infidelity, money, or domestic abuse. One day, Sonny sees a bruise on Connie's face and she tells him that Carlo hit her after she asked him if he was having an affair. Sonny tracks down and severely beats Carlo in the middle of a crowded street for brutalizing the pregnant Connie, and threatens to kill Carlo if he ever harms Connie again. An angry Carlo responds by plotting with Tattaglia and Don Emilio Barzini , the Corleones' chief rivals, to have Sonny killed. Later, Carlo has one of his mistresses phone his house, knowing that Connie will answer. The woman asks Connie to tell Carlo not to meet her tonight. The very pregnant and distraught Connie throws a tantrum, throwing the plates with their dinner around the dining room and kitchen. Carlo takes advantage of the altercation to beat Connie in order to lure Sonny out in the open and away from the Corleone compound. When Connie phones the compound to tell Sonny that Carlo has beaten her again, the enraged Sonny drives off (alone and unprotected) to fulfill his threat against Carlo. On the way to Connie and Carlo's house, Sonny is ambushed at a toll booth on the Long Island Causeway and violently shot to death by several carloads of hitmen wielding Thompson sub-machine guns. Tom Hagen relays the news of Sonny's massacre to the Don, who calls in the favor from Bonasera to personally handle the embalming of Sonny's body. Rather than seek revenge for Sonny's killing, Don Corleone meets with the heads of the Five Families to negotiate a cease-fire. Not only is the conflict draining all their assets and threatening their survival, but ending it is the only way that Michael can return home safely. Reversing his previous decision, Vito agrees that the Corleone family will provide political protection for Tattaglia's traffic in heroin, as long as it is controlled and not sold to children. At the meeting, Don Corleone deduces that Don Barzini, not Tattaglia, was ultimately behind the start of the mob war and Sonny's death, despite showing early signs of senility. In Sicily, Michael patiently waits out his exile, protected by Don Tommasino (Corrado Gaipa), an old family friend. Michael aimlessly wanders the countryside, accompanied by his ever-present bodyguards, Calo (Franco Citti) and Fabrizio (Angelo Infanti). In a small village, Michael meets and falls in love with Apollonia Vitelli (Simonetta Stefanelli), the beautiful young daughter of a bar owner. They court and marry in the traditional Sicilian fashion, but soon Michael's presence becomes known to Corleone enemies. One day, while Michael is teaching his new bride to drive, Tommasino brings the bad news about Sonny's assassination. He wants to movie Michael to a safer location. As the couple is about to leave, Apollonia is killed as a result of a rigged car (originally intended for Michael) exploding on ignition; Michael, who saw the car explode, spots Fabrizio hurriedly leaving the grounds seconds before the explosion, implicating him in the assassination plot. (In a deleted scene, Fabrizio is found years later and killed.) With his safety guaranteed, Michael returns home. More than a year later, in 1950, he reunites with his former girlfriend Kay after a total of four years of separation -- three in Italy and one in America. He tells her he wants them to be married. Although Kay is hurt that he waited so long to contact her, she accepts his proposal. With Don Vito semi-retired, Sonny dead, and middle brother Fredo considered incapable of running the family business, Michael is now in charge; he promises Kay he will make the family business completely legitimate within five years. Two years later, Clemenza and Salvatore Tessio (Abe Vigoda), complain that they are being pushed around by the Barzini Family and ask permission to strike back, but Michael denies the request. He plans to move the family operations to Nevada and after that, Clemenza and Tessio may break away to form their own families in the New York area. Michael further promises Connie's husband, Carlo, that he will be his right hand man in Nevada (Carlo had grown up there), unaware of his part in Sonny's assassination. Tom Hagen has been removed as consigliere and is now merely the family's lawyer, with Vito serving as consigliere. Privately, Hagen inquires about his change in status, and also questions Michael about a new regime of 'soldiers' secretly being built under Rocco Lampone (Tom Rosqui). Don Vito explains to Hagen that Michael is acting on his advice. Another year or so later, Michael travels to Las Vegas and meets with Moe Greene (Alex Rocco), a rich and shrewd casino boss looking to expand his business dealings. After the Don's attempted assassination, Fredo had been sent to Las Vegas to learn about the casino business from Greene. Michael arrogantly offers to buy out Greene but is rudely rebuffed. Greene believes the Corleones are weak and that he can secure a better deal from Barzini. As Moe and Michael heatedly negotiate, Fredo sides with Moe. After Moe storms out of the meeting, Michael warns Fredo to never again 'take sides with anyone against the family.' Michael returns home. In a private moment, Vito explains his expectation that the Family's enemies will attempt to murder Michael by using a trusted associate to arrange a meeting as a pretext for assassination. Vito also reveals that he had never really intended a life of crime for Michael, hoping that his youngest son would hold legitimate power as a senator or governor. Some months later, Vito collapses and dies while playing with his young grandson Anthony (Anthony Gounaris) in his tomato garden. At the burial, Tessio conveys a proposal for a meeting with Barzini, which identifies Tessio as the traitor that Vito was expecting. Kay asks Michael if he'll agree to be godfather to Connie and Carlo's newborn son. Michael agrees and seizes the opportunity to eliminate competition from the other five families while also using the baptism as an alibi. The murders occur simultaneously during the ceremony: Don Stracci (Don Costello) is gunned down along with his bodyguard in a hotel elevator by a shotgun-wielding Clemenza. Moe Greene is killed while having a massage, shot through the eye by an unidentified assassin. Don Cuneo (Rudy Bond) is trapped in a revolving door at the St. Regis Hotel and shot dead by soldier Willi Cicci (Joe Spinell). Don Tattaglia is assassinated in bed, along with a prostitute, by Rocco Lampone and an unknown associate. Don Barzini is killed on the steps of his office building along with his bodyguard and driver, shot by Al Neri (Richard Bright), disguised in his old police uniform. After the baptism, Tessio believes he and Hagen are on their way to the meeting between Michael and Barzini that he has arranged. Instead, he is surrounded by Willi Cicci and other button men as Hagen steps away. Realizing that Michael has uncovered his betrayal, Tessio tells Hagen that he always respected Michael, and that his disloyalty 'was only business.' He asks if Tom can get him off for 'old times' sake,' but Tom says he cannot. Tessio is driven away and never seen again (it is implied that Cicci shoots and kills Tessio with his own gun after he disarms him prior to entering the car). Meanwhile, Michael confronts Carlo about Sonny's murder and forces him to admit his role in setting up the ambush, having been approached by Barzini himself. (The hitmen who killed Sonny were the core members of Barzini's personal bodyguard.) Michael assures Carlo he will not be killed, but his punishment is exclusion from all family business. He hands Carlo a plane ticket to exile in Las Vegas. However, when Carlo gets into a car headed for the airport, he is immediately garroted to death by Clemenza, on Michael's orders. Later, a hysterical Connie confronts Michael at the Corleone compound as movers carry away the furniture in preparation for the family move to Nevada. She accuses him of murdering Carlo in retribution for Carlo's brutal treatment of her and for Carlo's suspected involvement in Sonny's murder and that Michael craftily waited until their father died so Vito couldn't stop him. After Connie is removed from the house, Kay questions Michael about Connie's accusation, but he refuses to answer, reminding her to never ask him about his business or what he does for a living. She insists, and Michael outright lies, reassuring his wife that he played no role in Carlo's death. Kay believes him and is relieved. The film ends with Clemenza and new caporegimes Rocco Lampone and Al Neri arriving and paying their respects to Michael. Clemenza kisses Michael's hand and greets him as 'Don Corleone.' As Kay watches, the office door is closed. A sole survivor tells of the twisty events leading up to a horrific gun battle on a boat, which began when five criminals met at a seemingly random police lineup. Following a truck hijack in New York, five criminals are arrested and brought together for questioning. As none of them are guilty, they plan a revenge operation against the police. The operation goes well, but then the influence of a legendary mastermind criminal called Keyser Söze is felt. It becomes clear that each one of them has wronged Söze at some point and must pay back now. The payback job leaves 27 men dead in a boat explosion, but the real question arises now: Who actually is Keyser Söze? On the deck of a ship in San Pedro, California, a figure identified as Keyser speaks with an injured man called Keaton. The two talk briefly, then Keyser appears to shoot Keaton before setting the ship ablaze. The next day, FBI Agent Jack Baer (Giancarlo Esposito) and U.S. Customs special agent Dave Kujan (Chazz Palminteri) arrive in San Pedro separately to investigate what happened on the boat. 27 people have been killed in the fire. There appear to be only two survivors: a crippled man named Roger 'Verbal' Kint (Kevin Spacey) and a hospitalized Hungarian criminal and crewman. Baer interrogates the Hungarian, who claims that Keyser Söze, a Turkish criminal mastermind with a nearly mythical reputation, was in the harbor 'killing many men.' The Hungarian begins to describe Söze while a translator interprets and a police sketch artist draws a rendering of Söze's face. Meanwhile, Verbal Kint tells the authorities everything he knows in exchange for immunity. After making his statement to the district attorney, Kint is placed in a police station office of Sgt. Jeff Rabin (Dan Hedaya), where both he and Kujan requests to hear the story again, from the beginning.... Verbal's tale starts six weeks earlier in New York City, when an armored truck containing spare gun parts is hijacked in Queens. The driver of the stolen truck was grabbed from behind and never saw the person or persons who accosted him, but the unknown hijacker spoke to him. Subsequently, the NYPD round up five different criminals: Dean Keaton (Gabriel Byrne) is a corrupt former police officer who has apparently given up his life of crime, and is arrested while eating dinner at a fancy restaurant; Michael McManus (Stephen Baldwin) is a crack shot with a temper and a wild streak, arrested in the middle of the night in his bed; Fred Fenster (Benicio Del Toro) is McManus's Hispanic partner who speaks in mangled English, and is nabbed on a street corner; Todd Hockney (Kevin Pollak) is a car mechanic and part-time hijacker who forms an instant rivalry with McManus, and is picked up while working on a car in his garage; and Verbal himself, who is a con artist that suffers from cerebral palsy. The five men are brought together for a lineup, and are told to say the phrase, 'Give me the keys, you fucking cocksucker.' They all manage to butcher the phrase. Later, while sitting in a holding cell, McManus convinces the others to join forces to commit a robbery targeting New York's Finest Taxi service, a group of corrupt NYPD police officers who escort smugglers to their destinations around the city. Their attack on one smuggler (Paul Bartel) goes off without a hitch, with farther reaching consequences as numerous other officers are indicted on corruption charges. With their loot, the quintet travels to California to sell it to McManus's fence, Redfoot (Peter Greene). Redfoot talks them into another job: robbing a purported jewel smuggler named Saul Bass. Instead of jewels or money, as they were told he was carrying, the smuggler had heroin. An angry confrontation between the thieves and Redfoot reveals that the job came from a lawyer named Kobayashi (Pete Postlethwaite). The five criminals later meet with Kobayashi, who claims to work for Keyser Söze and blackmails them into destroying the cargo of a ship coming to San Pedro harbor. In the present, Verbal Kint tells Kujan the story of Keyser Söze as he apparently heard it. Verbal's flashback reveals Söze's family being harassed by a rival Hungarian gang in Turkey. Söze goes on a murderous vendetta against all those who were involved. Afterwards, he apparently disappears; thereafter, he rarely conducts business without an alias and maintains anonymity between himself and anyone working for him. With time, Söze's story takes on mythic stature, with most people either doubting his existence or disbelieving it entirely. Back in the present, Kint continues to tell his version of what happened then. His flashback resumes to reveal Fenster's attempt to run away, but he is tracked and killed (off-camera) by Kobayashi. The remaining four thieves kidnap Kobayashi, believing Söze to be a cover for his activities and intending to kill him if he does not leave them alone. Before McManus can execute him, Kobayashi reveals that lawyer Edie Finneran (Suzy Amis), Keaton's girlfriend, is in his office. Kobayashi also says that he has the will, the information, and the means to injure or kill the remaining four criminals' relatives if they do not go through with the arrangement. On the night of the cocaine deal, the sellers--a group of Argentine mobsters--are on the dock, as are the buyers--a group of Hungarian mobsters. Keaton tells Verbal to stay back, and to take the money to Edie if the plan goes awry so she can pursue Kobayashi 'her way;' Verbal reluctantly agrees to do so. He watches the boat from a distance, in hiding, as Keaton, McManus and Hockney attack the men at the pier in a huge gunfight. During the battle, Hockney is killed as Keaton and McManus discover separately that there is no cocaine on the boat. Meanwhile, the Hungarians, yet untouched by the thieves, are being picked off by an unseen killer, and a closely-guarded Hispanic passenger is killed by an unseen assailant. McManus is killed with a knife to the back of his neck, and Keaton, turning away to leave, is shot in the back. A figure in a dark coat appears, presumably Keyser Söze, and lights a cigarette with a gold lighter. He appears to speak briefly with Keaton before apparently shooting him (the scene which began the film in medias res). In the present, with Verbal's story finished, Kujan reveals what he has deduced, with the aide of Agent Baer: The boat hijacking was not about cocaine, but rather to ensure that one man aboard the ship--the Hispanic passenger held captive, named Arturo (Castulo Guerra), one of the few individuals alive who could positively identify Söze--is killed. After Söze presumably killed the man, he eliminated everyone else on the ship and set it ablaze; Kujan reveals that Edie has also been killed. Kujan has concluded that Keaton was Keyser Söze. Verbal admits that the whole affair, from the beginning, was Keaton's idea. His bail having been posted, Verbal departs with his immunity. Verbal retrieves his personal effects from the property officer as Kujan, relaxing in Rabin's office that he used for the interrogation, notices that details and names from Verbal's story are words appearing on various objects around the room. 'Redfoot' is the name on a wanted poster, for example, and 'Kobayashi' is written on the bottom of a coffee cup that Verbal handled earlier. Quickly putting the pieces together, Kujan realizes that Verbal made up practically the entire story as he talked. He runs outside just as a fax arrives with the police artist's impression of Keyser Söze's face.... which resembles Verbal Kint. As he leaves the police station, Verbal's distinctive limp gradually disappears and he shakes out his palsied hand. He steps into a waiting car driven by 'Mr. Kobayashi,' departing just as Kujan comes outside to see too late that Kint is gone. Verbal Kint is Keyser Söze, so how much of his narration was truthful is open to interpretation. In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis. The relocation of Polish Jews from surrounding areas to Krakow begins in late 1939, shortly after the outbreak of World War II, when the German Army defeats the Polish Army in three weeks. Oskar Schindler , a successful businessman, arrives from Czechoslovakia in hopes of using the abundant cheap labour force of Jews to manufacture enamelware for the German military. Schindler, an opportunistic member of the Nazi party, lavishes bribes upon the army and SS officials in charge of procurement. Sponsored by the military, Schindler acquires a factory for the production of army mess kits and cooking paraphernalia. Not knowing much about how to properly run such an enterprise, he gains a contact in Itzhak Stern ,a functionary in the local Judenrat (Jewish Council) who has contacts with the now-underground Jewish business community in the ghetto. They loan him the money for the factory in return for a small share of products produced (for trade on the black market). Opening the factory, Schindler pleases the Nazis and enjoys his new-found wealth and status as 'Herr Direktor,' while Stern handles all administration. Stern suggests Schindler hire Jews instead of Poles because they cost less (the Jews themselves get nothing; the wages are paid to the Reich). Workers in Schindler's factory are allowed outside the ghetto, and Stern falsifies documents to ensure that as many people as possible are deemed 'essential' by the Nazi bureaucracy, which saves them from being transported to concentration camps, or even being killed. Amon Göth arrives in Krakow to initiate construction of a labor camp nearby, Paszów. The SS soon liquidates the Krakow ghetto, sending in hundreds of troops to empty the cramped rooms and shoot anyone who protests, is uncooperative, elderly, or infirmed, or for no reason at all. Schindler watches the massacre from the hills overlooking the area, and is profoundly affected. He nevertheless is careful to befriend Göth and, through Stern's attention to bribery, he continues to enjoy the SS's support and protection. The camp is built outside the city at Paszów. During this time, Schindler bribes Göth into allowing him to build a sub-camp for his workers, with the motive of keeping them safe from the depredations of the guards. Eventually, an order arrives from Berlin commanding Göth to exhume and destroy all bodies of those killed in the Krakow ghetto, dismantle Paszów, and to ship the remaining Jews to Auschwitz. Schindler prevails upon Göth to let him keep 'his' workers so that he can move them to a factory in his old home of Zwittau-Brinnlitz, in Moravia -- away from the 'final solution' now fully under way in occupied Poland. Göth acquiesces, charging a certain amount for each worker. Schindler and Stern assemble a list of workers that should keep them off the trains to Auschwitz. 'Schindler's List' comprises these 'skilled' inmates, and for many of those in Paszów, being included means the difference between life and death. Schindler also plays a game of high card draw for one worker in particular, Helen Hirsch, who'd been serving as Göth's housekeeper and had been a victim of his continual abuse. Göth is reluctant, hoping to run away with her but knowing that such an action would result in his death as well as hers. He also floats the idea of simply executing her but finally decides to play Schindler for Helen's life. Helen is among those who board the train to Brinnlitz. All of the men on Schindler's list arrive safely at the new site, with the exception to the train carrying the women and the children, which is accidentally redirected to Auschwitz. There, the women are directed to what they believe is a gas chamber; after a harrowing experience where their hair is crudely cut off and they are forced to strip, they see only water falling from the showers. The day after, the women are shown waiting in line for work. In the meantime, Schindler had rushed immediately to Auschwitz to solve the problem and to get the women out of Auschwitz; to this end he bribes the camp commander, Rudolf Höss with a cache of diamonds so that he is able to spare all the women and the children. However, a last problem arises just when all the women are boarding the train because several SS officers attempt to hold some children back and prevent them from leaving. The novel is told in epistolary format, as a series of letters, diary entries, ships' log entries, and so forth. The main writers of these items are also the novel's protagonists. The story is occasionally supplemented with newspaper clippings that relate events not directly witnessed by the story's characters. The tale begins with Jonathan Harker, a newly qualified English solicitor, journeying by train and carriage from England to Count Dracula's crumbling, remote castle (situated in the Carpathian Mountains on the border of Transylvania, Bukovina and Moldavia). The purpose of his mission is to provide legal support to Dracula for a real estate transaction overseen by Harker's employer, Peter Hawkins, of Exeter in England. At first enticed by Dracula's gracious manner, Harker soon discovers that he has become a prisoner in the castle. He also begins to see disquieting facets of Dracula's nocturnal life. One night while searching for a way out of the castle, and against Dracula's strict admonition not to venture outside his room at night, Harker falls under the spell of three wanton female vampires, 'the Sisters.' He is saved at the last second by the Count, because he wants to keep Harker alive just long enough to obtain needed legal advice and teachings about England and London (Dracula's planned travel destination was to be among the 'teeming millions'). Harker barely escapes from the castle with his life. Not long afterward, a Russian ship, the Demeter, having weighed anchor at Varna, runs aground on the shores of Whitby, England, during a fierce tempest. All of the crew are missing and presumed dead, and only one body is found, that of the captain tied to the ship's helm. The captain's log is recovered and tells of strange events that had taken place during the ship's journey. These events led to the gradual disappearance of the entire crew apparently owing to a malevolent presence on board the ill-fated ship. An animal described as a large dog is seen on the ship leaping ashore. The ship's cargo is described as silver sand and boxes of 'mould', or earth, from Transylvania. Soon Dracula is tracking Harker's devoted fiancée, Wilhelmina 'Mina' Murray, and her friend, Lucy Westenra. Lucy receives three marriage proposals in one day, from Dr. John Seward; Quincey Morris; and the Hon. Arthur Holmwood (later Lord Godalming). Lucy accepts Holmwood's proposal while turning down Seward and Morris, but all remain friends. There is a notable encounter between Dracula and Seward's patient Renfield, an insane man who means to consume insects, spiders, birds, and other creatures &mdash; in ascending order of size &mdash; in order to absorb their 'life force'. Renfield acts as a motion sensor, detecting Dracula's proximity and supplying clues accordingly. Lucy begins to waste away suspiciously. All of her suitors fret, and Seward calls in his old teacher, Professor Abraham Van Helsing from Amsterdam. Van Helsing immediately determines the cause of Lucy's condition but refuses to disclose it, knowing that Seward's faith in him will be shaken if he starts to speak of vampires. Van Helsing tries multiple blood transfusions, but they are clearly losing ground. On a night when Van Helsing must return to Amsterdam (and his message to Seward asking him to watch the Westenra household is delayed), Lucy and her mother are attacked by a wolf. Mrs. Westenra, who has a heart condition, dies of fright, and Lucy apparently dies soon after. Lucy is buried, but soon afterward the newspapers report children being stalked in the night by a 'bloofer lady' (as they describe it), i.e. 'beautiful lady'. Van Helsing, knowing that this means Lucy has become a vampire, confides in Seward, Lord Godalming and Morris. The suitors and Van Helsing track her down, and after a disturbing confrontation between her vampiric self and Arthur, they stake her heart, behead her, and fill her mouth with garlic. Around the same time, Jonathan Harker arrives home from recuperation in Budapest (where Mina joined and married him after his escape from the castle); he and Mina also join the coalition, who turn their attentions to dealing with Dracula. After Dracula learns of Van Helsing's and the others' plot against him, he takes revenge by visiting – and feeding from – Mina at least three times. Dracula also feeds Mina his blood, creating a spiritual bond between them to control her. The only way to forestall this is to kill Dracula first. Mina slowly succumbs to the blood of the vampire that flows through her veins, switching back and forth from a state of consciousness to a state of semi-trance during which she is telepathically connected with Dracula. This telepathic connection is established to be two-way, in that the Count can influence Mina, but in doing so betrays to her awareness of his surroundings. After the group sterilizes all of his lairs in London by putting pieces of consecrated host in each box of earth, Dracula flees back to his castle in Transylvania, transported in a box with transfer and portage instructions forwarded, pursued by Van Helsing's group, who themselves are aided by Van Helsing hypnotizing Mina and questioning her about the Count. The group splits in three directions. Van Helsing goes to the Count's castle and kills his trio of brides, and shortly afterwards all converge on the Count just at sundown under the shadow of the castle. Harker and Quincey rush to Dracula's box, which is being transported by Gypsies. Harker shears Dracula through the throat with a Kukri while the mortally wounded Quincey, slashed by one of the crew, stabs the Count in the heart with a Bowie knife. Dracula crumbles to dust, and Mina is freed from his curse. The book closes with a note about Mina's and Jonathan's married life and the birth of their first-born son, whom they name after all four members of the party, but refer to only as Quincey in remembrance of their American friend. The plot follows Harvard symbologist Robert Langdon, as he tries to stop the Illuminati, a legendary secret society, from destroying Vatican City with the newly discovered power of antimatter. CERN director Maximilian Kohler discovers one of the facility's physicists, Leonardo Vetra, murdered. His chest is branded with an ambigram of the word 'Illuminati'. Kohler contacts Robert Langdon, an expert on the Illuminati, who determines that the ambigram is authentic. Kohler calls Vetra's adopted daughter Vittoria to the scene, and it is ascertained that the Illuminati have stolen a canister containing antimatter — a substance with destructive potential comparable to a small nuclear weapon. When at CERN the canister is stored in a unique electrical charger which ensures the anti-matters stability but when removed its back-up battery provides power for 24 hours after which the anti-matter will self-destruct. The canister is somewhere in Vatican City, with a security camera in front of it, as its digital clock counts down to the explosion. Langdon and Vittoria make their way to Vatican City, where the Pope has recently died. It is discovered that the four Preferiti, cardinals who are the most likely papal successor, are missing. Langdon and Vittoria search for the Preferiti in hopes that they will also find the antimatter canister. Their search is assisted by Camerlengo Carlo Ventresca (the late pope's closest aide) and the Vatican's Swiss Guard. Langdon attempts to retrace the steps of the 'Path of Illumination', a process once used by the Illuminati as a means of inducting new members; aspirants to the order were required to follow a series of subtle clues left in various landmarks in and around Rome. The clues indicate the secret meeting place of the Illuminati. Langdon sets off on the Path of Illumination in hopes of delivering the Preferiti and recovering the antimatter canister. The Path leads Langdon to four locations in Rome, each associated with one of the primordial elements: 'Earth', 'Air', 'Fire', and 'Water'. Langdon finds one of the Preferiti murdered in a way thematically related to each location's related element. The first cardinal was branded with an Earth ambigram and had soil forced down his throat; the second was branded with an air ambigram and had his lungs punctured; the third was branded with a fire ambigram and was burned alive; and the fourth was branded with a water ambigram and was left to drown at the bottom of a fountain. After finding the bodies of the first two Preferiti, Langdon hurries to the Santa Maria della Vittoria Basilica and finds the Preferiti's abductor in the act of setting the third cardinal on fire. The kidnapper is an unnamed assassin who is working under the orders of the Illuminati master 'Janus', whose true identity is unknown. Commander Olivetti is killed and the assassin kidnaps Vittoria. Langdon escapes and accosts the assassin at the final element's landmark (Water), but is unable to save the cardinal. Langdon must complete the Path of Illumination in order to find the assassin and rescue Vittoria. His search leads him to Castel Sant'Angelo, that hides a tunnel leading directly into the pope's chambers in the Vatican. Langdon frees Vittoria, and together they send the assassin falling several hundred feet to his death. The two hurry back to St. Peter's Basilica, where they find that Kohler has arrived to confront the camerlengo in private. Langdon and Vittoria fear that Kohler is Janus, and that he has come to murder the camerlengo. Hearing the camerlengo scream in agony from being branded with the Illuminati Diamond, the Swiss Guards burst into the room and open fire on Kohler. Just before he dies, Kohler gives Langdon a videotape that he claims will explain everything. With time running out, the Swiss Guard evacuates the Basilica. The camerlengo rushes back in, claiming that he has received a vision revealing the location of the antimatter canister. With Langdon in pursuit, the camerlengo ventures into the catacombs and finds the canister sitting atop the tomb of Saint Peter. Langdon and the camerlengo retrieve the antimatter and get in a helicopter with only minutes to spare. The camerlengo manages to parachute safely onto the roof of St. Peter's just as the canister explodes harmlessly in the sky. The crowd in St. Peter's Square look in awe as the camerlengo stands triumphantly before them. Because of this 'miracle', the papal conclave debate whether to elect the camerlengo as the new Pope. Langdon managed to survive the explosion by using a window cover from the helicopter as a parachute, and landed in the Tiber River. After viewing Kohler's tape Langdon, Vittoria, and the cardinals confront the camerlengo; Shortly before the beginning of the novel, the Pope met with Leonardo Vetra who believed that anti-matter was capable of establishing a link between Man and God. Vetra's beliefs caused great discomfort to the camerlengo. While discussing Vetra, the pope reveals that his support is due to science having given him a son. Without waiting to hear the explanation (that the child was the result of artificial insemination), and horrified that the Pope appeared to have broken his vow of chastity, the camerlengo plots to 'rectify' the situation. He poisoned the pope and, under the guise of an Illuminati master (Janus), he recruited the assassin, to kill Vetra, steal the antimatter, and kidnap and murder the Preferiti. The Camerlengo planted the antimatter in St. Peter's in order to be seen as the savior of Christendom. The Illuminati 'involvement' was merely a plot engineered by the camerlengo to cover his own plans. It is revealed that Camerlengo Ventresca himself was the birth son of the late pope, conceived through artificial insemination. Overcome with guilt Ventresca soaks himself in oil and immolates himself before a crowd of onlookers in St. Peter's Square. A group of scientists, including psychologist Norman Johnson, mathematician Harry Adams, biologist Beth Halpern, and astrophysicist Ted Fielding, along with U.S. Navy personnel, are dispatched to a deep sea habitat at the bottom of the Pacific Ocean to explore a crashed spacecraft. To their surprise, they discover the spacecraft is not alien, but an American spacecraft constructed in the future and apparently sent through time, crashing 350 years before its creation. On further exploration, the team discovers a mysterious spherical artifact, clearly of extraterrestrial origin, which quickly becomes the focus of their attention. Harry becomes quite certain that, because the ship's future builders didn't seem to learn that their ship had already been discovered, the members of the team aren't likely to survive. At this point, a storm traps the scientists on the ocean floor without contact or support from the surface for over a week. The crew soon focuses on asking questions about the sphere and then on attempting to open it and learn about its nature, contents, and origin. Harry eventually succeeds in opening it and goes inside. Upon returning, he has a terrible headache and he remembers little about what happened inside or how he opened it. The scientists are eventually contacted by an intelligent, seemingly-friendly lifeform which calls itself Jerry, apparently from within the sphere. It first contacts them via a numeric code, which Harry translates. But while they struggle to communicate with Jerry, increasingly bizarre and deadly events occur, including the appearance of sea creatures that Beth claims don't exist. Jerry tells them he is 'manifesting' the creatures. Members of the team start to die in various attacks by sea life, and the dwindling survivors struggle to placate the unthinkably powerful, childlike, and temperamental Jerry. Norman suddenly has an important role when he realizes he must use psychology to keep the surviving team (now only himself, Beth, and Harry) alive by placating Jerry. Translating the original code himself, though, Norman discovers that Jerry is actually Harry: by entering the sphere, Harry acquired the power to manifest his subconscious thoughts into reality. As Harry noted his childhood fears of squids and Twenty Thousand Leagues Under the Sea, he has unconsciously created them as an enemy. Beth and Norman tranquilize Harry with a powerful mixture of sedatives and wait for contact to be re-established with the surface. However, although Harry is sedated, the manifestations continue. Beth accuses Norman of having entered the sphere and gaining access to the power. Though unable to recall this incident, Norman is close to yielding until he watches a security video of Beth entering the sphere herself. Concluding that Norman is a threat to her, Beth irrationally plants potent explosives around the spacecraft and habitat and then attempts to suffocate Norman with the habitat's climate systems. Norman escapes to the spacecraft and, figuring out at last how to open it, enters the sphere. Norman begins to ascend by himself in the submarine, but realizes that he could never leave the others to die. Now with the same power of thought as Harry and Beth, Norman fights Beth and brings both her and Harry to the escape submarine before the explosives destroy the site. Afterward, while in a surface decompression chamber, the three survivors ponder what to tell the Navy about what happened. Realizing they could not control the power, they decide to use the power to remove it from themselves and their memories simultaneously, replacing it with memories of a technical failure. Afterwards, as they mourn the colleagues lost to this scenario, Norman compliments Beth's appearance, saying that she looks lovely despite their hardship in the deep. Beth only smiles. Louvre curator and Priory of Sion Grand Master Jacques Saunière is fatally shot one night at the museum by a man named Silas, who is working on behalf of someone known only as the Teacher, who wishes to discover the location of the 'keystone,' an item crucial to the search for the Holy Grail. After Saunière's body is discovered in the pose of the Vitruvian Man, the police summon Harvard Professor Robert Langdon, who is in town on business. Police Captain Bezu Fache tells him that he was summoned to help the police decode the cryptic message Saunière left during the final minutes of his life. The note also includes a Fibonacci sequence, left as a code. Langdon explains to Fache that Saunière was a leading authority in the subject of goddess artwork and that the pentacle Saunière drew in his own blood represents an allusion to the goddess and not 'devil worship', as Fache believes. A police cryptographer, Sophie Neveu secretly explains to Langdon she is Saunière's estranged granddaughter, and that Fache thinks Langdon is the murderer, because of the note her grandfather left saying to 'find Robert Langdon,' which she says Fache had erased prior to Langdon's arrival. Sophie is troubled by memories of her grandfather's involvement in a secret pagan group. However, she understands that her grandfather intended Langdon to decipher the code, which she and Langdon realize leads them to a safe deposit box at the Paris branch of the Depository Bank of Zurich, which Sophie and Langdon go to after escaping the police. In the safe deposit box they find the keystone: a cryptex, a cylindrical, hand-held vault with five concentric, rotating dials labeled with letters that when lined up properly form the correct password, unlocking the device. If the cryptex is forced open, an enclosed vial of vinegar ruptures and dissolves the message, which was written on papyrus. The box containing the cryptex contains clues to its password. Langdon and Neveu take the keystone to Langdon's friend, Sir Leigh Teabing, an expert on the Holy Grail. There, Teabing explains that the Grail is not a cup, but the tomb containing the bones of Mary Magdalene. The group then flees the country in Teabing's private plane, on which they conclude that the proper combination of letters spell out Sophie's given name, 'SOFIA.' Opening the cryptex, they discover a smaller cryptex inside it, along with another riddle that ultimately leads the trio to the tomb of Isaac Newton at Westminster Abbey. During the flight to Britain, Sophie reveals the source of her estrangement from her grandfather, ten years earlier. Arriving home unexpectedly from college, Sophie secretly witnesses a spring fertility rite conducted in the basement of her grandfather's country estate. From her hiding place, she is shocked to see her grandfather making love to a woman at the center of a ritual attended by men and women who are wearing masks and chanting praise to the goddess. She flees the house and breaks off all contact with Saunière. Langdon explains that what she witnessed was an ancient ceremony known as Hieros gamos or 'sacred marriage'. By the time they arrive at Westminster Abbey, Teabing is revealed to be the Teacher for whom Silas is working. Teabing wishes to use the Holy Grail, which he believes is a series of documents establishing that Jesus Christ married Mary Magdalene and bore children, in order to ruin the Vatican. He compels Langdon at gunpoint to solve the second cryptex's password, which Langdon realizes is 'APPLE.' Langdon secretly opens the cryptex and removes its contents before destroying it in front of Teabing. Teabing is arrested by Fache, who by now knows that Langdon was innocent. Bishop Aringarosa, realizing that Silas has been used to murder innocent people rushes to help the police find him. Silas is found hiding in an Opus Dei Center, when he realizes that the police have found him, which causes him to rush out and accidentally shoot Bishop Aringarosa. Bishop Aringarosa survives but is informed that Silas was found dead later, apparently by suicide. The final message inside the second keystone leads Sophie and Langdon to Rosslyn Chapel, whose docent turns out to be Sophie's long-lost brother, whom Sophie had been told died as a child in the car accident that killed her parents. The guardian of Rosslyn Chapel, Marie Chauvel, is Sophie's long-lost grandmother, and the widow of Jacques Saunière. It is revealed that Sophie is a descendant of Jesus Christ and Mary Magdalene. The Priory of Sion hid her identity to protect her from possible threats to her life. The real meaning of the last message is that the Grail is buried beneath the small pyramid directly below the inverted glass pyramid of the Louvre. It also lies beneath the 'Rose Line,' an allusion to 'Roslyn.' Langdon figures out this final piece to the puzzle in the last pages of the book, but he does not appear inclined to tell anyone about this. ")
df = pd.DataFrame(dict(r=predictions, theta=['E/I', 'N/S', 'T/F', 'P/J']))
#fig = px.line_polar(df, r='r', theta='theta', line_close=True)
#fig.show()
print('User 2542의 Personality는: ', predictions)

User 2542의 Personality는:  ['E', 'S', 'F', 'J']


# **User 2542의 Personality에 맞는 게임 추천**

In [24]:
MBTI=(data3['MBTI']=="['E', 'S', 'F', 'J']")
data3[MBTI]

,All_good_reviews_ratio,MBTI,game_description
name,,,
Dodge Bubble,100,"['E', 'S', 'F', 'J']",About This Game Dodge bubble is a casual sur...
Hiveswap Friendsim - Volume Five,100,"['E', 'S', 'F', 'J']",About This Content Coming in hot and fresh fr...
Grow Home Soundtrack DLC,100,"['E', 'S', 'F', 'J']","About This Content Sit back, relax and soak i..."
Cloudbuilt - Defiance,100,"['E', 'S', 'F', 'J']",About This Content Explore a new path in Demi...
ETHEREAL,100,"['E', 'S', 'F', 'J']",About This Game ETHEREAL is a profound exper...
...,...,...,...
LEGO® Harry Potter: Years 1-4,76,"['E', 'S', 'F', 'J']",About This Game Build the adventure from Priv...
Radiation Island,73,"['E', 'S', 'F', 'J']",About This Game Radiation Island is a surviva...
Revhead,73,"['E', 'S', 'F', 'J']",About This Game Revhead is a car racing simul...


In [25]:
user2542 = pd.DataFrame(
    data={'User ID':['2542', '', '', ''],
          'MBTI':['ESFJ','', '', ''],
          'Movie':['Godfather, The(1972)','Usual Suspects, The(1995)',"Schindler's List(1993)",'Rear Window(1954)'],
          'Book':['Angels and Demons','Dracula','The Da vinci Code','Sphere'],
          'Game':['Dodge Bubble','Hiveswap Friendsim - Volume Five','Grow Home Soundtrack DLC','Cloudbuilt - Defiance']
          },
    columns=['User ID','MBTI','Movie','Book','Game']
) 

# **최종적으로 user2542에게 추천된 Movie, Book, Game 목록**

In [26]:
user2542

,User ID,MBTI,Movie,Book,Game
0,2542,ESFJ,"Godfather, The(1972)",Angels and Demons,Dodge Bubble
1,,,"Usual Suspects, The(1995)",Dracula,Hiveswap Friendsim - Volume Five
2,,,Schindler's List(1993),The Da vinci Code,Grow Home Soundtrack DLC
3,,,Rear Window(1954),Sphere,Cloudbuilt - Defiance
